In [1]:
%load_ext sql
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [2]:
%%sql
ATTACH 'CourseData.db' AS CDDB

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [3]:
%%sql
select * from CDDB.PROGRAMS LIMIT 1

 * sqlite:///CourseDataWarehouse.db
Done.


PID,program_code,program_name
1,AN,Asian Studies


In [4]:
%%sql

DROP TABLE IF EXISTS PROGRAMS_DIM;
DROP TABLE IF EXISTS LOCATIONS_DIM;
DROP TABLE IF EXISTS TIMES_DIM;
DROP TABLE IF EXISTS FACULTY_DIM;
DROP TABLE IF EXISTS COURSE_CATALOGS_DIM;
DROP TABLE IF EXISTS COURSES_FACT;





vacuum; 

CREATE TABLE PROGRAMS_DIM (
    PID INTEGER NOT NULL PRIMARY KEY,
    program_code TEXT NOT NULL,
    program_name TEXT NOT NULL
);


CREATE TABLE LOCATIONS_DIM (
    LID INTEGER PRIMARY KEY,
    location TEXT ,
    BuildingName TEXT

);


CREATE TABLE FACULTY_DIM (
    FID  INTEGER NOT NULL PRIMARY KEY,
    instructor_fname TEXT NOT NULL,
    instructor_lname TEXT NOT NULL
);


CREATE TABLE COURSE_CATALOGS_DIM (
    CID INTEGER PRIMARY KEY,

    CatalogYear TEXT NULL,
    Credits TEXT  NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees INTEGER,
    Description TEXT,
    Attributes TEXT
);



CREATE TABLE COURSES_FACT (
    CID INTEGER NULL ,
    CRN INTEGER NOT NULL,
    SEMESTER TEXT NOT NULL ,
    YEAR INTEGER NOT NULL,
    Title TEXT  NULL,
    SECTION TEXT NOT NULL,
    Act INTEGER,
    Cap INTEGER,
    Rem INTEGER, 
    timecodes TEXT,
    PID INTEGER  NULL,
    LID INTEGER  NULL,
    FID INTEGER  NULL,
   Catalog_ID  TEXT NULL ,
    FOREIGN KEY (PID) REFERENCES PROGRAMS_DIM (PID),
    FOREIGN KEY (LID) REFERENCES LOCATIONS_DIM (LID),
    FOREIGN KEY (FID) REFERENCES FACULTY_DIM (FID),
    FOREIGN KEY (CID) REFERENCES COURSE_CATALOGS_DIM (CID)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [6]:
%%sql 


INSERT INTO PROGRAMS_DIM ( program_name, program_code)
SELECT DISTINCT  program_name, program_code
FROM CDDB.PROGRAMS;


INSERT INTO LOCATIONS_DIM (location ,BuildingName )
SELECT  location , BuildingName
FROM CDDB.LOCATION;


INSERT INTO FACULTY_DIM (instructor_fname , instructor_lname)
SELECT  DISTINCT instructor_fname , instructor_lname
FROM CDDB.FACULTY;


INSERT INTO COURSE_CATALOGS_DIM (CatalogYear, credits, Prereqs, Coreqs, Fees, Description, Attributes)
SELECT CDDB.COURSE.CatalogYear , credits, prereqs, coreqs, fees, description, attributes
FROM CDDB.COURSE;



INSERT INTO COURSES_FACT (Catalog_ID , CRN, SEMESTER , YEAR , TITLE, SECTION, ACT, CAP,REM, TIMECODES, PID, LID, FID, CID)
SELECT DISTINCT CDDB.COURSE_OFFERING.Catalog_ID, CDDB.CLASS_MEETINGS.CRN, CDDB.CLASS_MEETINGS.SEMESTER , CDDB.CLASS_MEETINGS.YEAR, CDDB.COURSE_OFFERING.title, CDDB.COURSE_OFFERING.SECTION, act, cap, rem, timecodes, PROGRAMS_DIM.PID, LOCATIONS_DIM.LID, FID,CDDB.COURSE.CID
FROM CDDB.COURSE_OFFERING
     JOIN CDDB.CLASS_MEETINGS USING  (COID)
    left JOIN LOCATIONS_DIM USING (LID)
     left JOIN CDDB.COURSE USING (CID)
    left JOIN PROGRAMS_DIM USING (PID)
    left JOIN FACULTY_DIM USING (FID)
    left JOIN COURSE_CATALOGS_DIM USING (CID);
    
    

    
   

 * sqlite:///CourseDataWarehouse.db
92 rows affected.
207 rows affected.
1104 rows affected.
6696 rows affected.
13756 rows affected.


[]